# Explore here

In [11]:
# Your code here
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")
data_uni = data.drop_duplicates()
data_uni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [12]:
data.to_csv("../data/raw/data.csv",index=False)

In [13]:
data_uni.describe()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
count,3140.000000,3.140000e+03,3.140000e+03,3140.000000,3.140000e+03,3140.000000,3.140000e+03,3140.000000,3.140000e+03,3140.000000,...,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000
mean,30401.640764,1.041894e+05,1.274030e+04,11.871051,1.336798e+04,12.694609,1.446933e+04,12.283979,1.391649e+04,11.751535,...,5827.242357,13.073503,12.088089,14.053726,9326.577707,3.446242,3.207516,3.710478,2466.234076,4.635350
std,15150.559265,3.335834e+05,4.180730e+04,2.124081,4.228439e+04,1.815044,4.957773e+04,3.126297,4.899095e+04,1.696599,...,15720.551934,2.724351,2.622948,2.824828,29754.601185,0.568059,0.527740,0.613069,7730.422067,1.510447
min,1001.000000,8.800000e+01,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,1.100000e+01,6.092789,...,7.000000,6.100000,5.500000,6.700000,11.000000,1.800000,1.700000,1.900000,3.000000,1.000000
25%,18180.500000,1.096325e+04,1.280500e+03,10.594639,1.374500e+03,11.674504,1.263750e+03,10.496774,1.232750e+03,10.689322,...,815.000000,11.200000,10.300000,12.100000,1187.750000,3.100000,2.900000,3.300000,314.750000,3.000000
50%,29178.000000,2.580050e+04,3.057000e+03,11.802727,3.274000e+03,12.687422,3.108000e+03,11.772649,3.000500e+03,11.580861,...,1963.500000,12.800000,11.800000,13.800000,2743.000000,3.400000,3.200000,3.700000,718.000000,5.000000
75%,45081.500000,6.791300e+04,8.097000e+03,12.951840,8.822250e+03,13.659282,8.976250e+03,13.182260,8.314250e+03,12.639379,...,4727.000000,14.800000,13.700000,15.900000,6679.250000,3.800000,3.500000,4.100000,1776.250000,6.000000
max,56045.000000,1.010552e+07,1.208253e+06,25.460677,1.239139e+06,23.304372,1.557073e+06,37.570198,1.501844e+06,22.225129,...,434075.000000,25.600000,24.200000,27.000000,952335.000000,6.200000,5.800000,6.600000,237766.000000,6.000000


In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

col_obj = data_uni.select_dtypes(include='object').columns.tolist()
col_num = data_uni.select_dtypes(exclude='object').columns.tolist()

def eliminar_atipicos(datos, columnas):
    new_data = datos
    for i in columnas:
        q1=new_data[i].quantile(0.25)
        q3=new_data[i].quantile(0.75)
        iqr = q3-q1
        low_lim = q1 - 1.5*iqr
        hi_lim = q3 + 1.5*iqr
        new_data = new_data[(new_data[i] <= hi_lim) & (new_data[i] >= low_lim)]
    return new_data.copy()

data_num_clean = eliminar_atipicos(data_uni,col_num)
# fig, axis = plt.subplots(figsize = (106,106))
# sns.heatmap(data_num_clean[col_num].corr(), annot = True, fmt = ".2f")

In [23]:
data_num_clean

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
14,1029,14987,1829,12.203910,1905,12.711016,1687,11.256422,1665,11.109628,...,1332,14.4,13.2,15.6,1665,3.6,3.3,3.9,416,6
146,5075,16434,1930,11.743945,2164,13.167823,2118,12.887915,1787,10.873798,...,1388,13.7,12.7,14.8,1759,3.6,3.3,3.9,461,6
163,5109,10673,1201,11.252694,1327,12.433243,1172,10.980980,1142,10.699897,...,928,14.4,13.4,15.6,1212,3.7,3.4,4.0,312,6
397,13025,18897,2359,12.483463,2509,13.277240,2310,12.224163,2169,11.478012,...,1773,14.7,13.6,15.7,2125,3.8,3.5,4.1,546,4
552,16007,6050,877,14.495868,886,14.644628,580,9.586777,682,11.272727,...,351,12.1,11.1,13.2,538,3.4,3.2,3.7,150,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2550,48059,13994,1654,11.819351,1772,12.662570,1411,10.082893,1636,11.690725,...,1002,13.6,12.5,14.7,1470,3.5,3.3,3.8,383,4
2559,48077,10456,1041,9.956006,1271,12.155700,998,9.544759,1070,10.233359,...,771,13.5,12.3,14.6,1125,3.5,3.3,3.8,294,4
2567,48093,13534,1602,11.836855,1701,12.568346,1392,10.285208,1336,9.871435,...,1010,15.0,13.8,16.2,1576,3.9,3.6,4.2,410,6
2587,48133,18322,2095,11.434341,2529,13.803078,2135,11.652658,1949,10.637485,...,1490,14.9,13.8,15.9,2141,3.9,3.6,4.2,564,6


In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
norm_features = scaler.fit_transform(data_num_clean[col_num])
data_scal = pd.DataFrame(norm_features, index = data_num_clean.index, columns = col_num)
data_scal["Heart disease_number"] = data_num_clean["Heart disease_number"]

correlation_matrix = data_scal.corr(numeric_only=True)
cor_target = correlation_matrix['Heart disease_number'].abs().sort_values(ascending=False)
top_features = cor_target[cor_target > 0.5]  # puedes cambiar el umbral
print(top_features[:20])
data_top = data_scal[top_features[:20].index]
X = data_top.drop(columns=["Heart disease_number"])
Y = data_scal["Heart disease_number"]

Heart disease_number                1.000000
CKD_number                          0.995595
diabetes_number                     0.993431
anycondition_number                 0.986984
70-79                               0.975316
COPD_number                         0.971983
county_pop2018_18 and older         0.971424
POP_ESTIMATE_2018                   0.968757
TOT_POP                             0.968757
Total Population                    0.967354
Obesity_number                      0.966409
White-alone pop                     0.965582
50-59                               0.965574
60-69                               0.964213
High school diploma only 2014-18    0.963376
40-49                               0.962366
Population Aged 60+                 0.959017
20-29                               0.955017
19-Oct                              0.952388
30-39                               0.948858
Name: Heart disease_number, dtype: float64


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import mean_squared_error, r2_score

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=8)

model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2_score(y_test,y_pred)

0.9295145533091584

In [27]:
x_train.to_csv("../data/processed/x_train.csv", index=False)
x_test.to_csv("../data/processed/x_test.csv", index=False)
y_train.to_csv("../data/processed/y_train.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)

In [28]:
from sklearn.linear_model import LogisticRegression, Lasso
las_model = Lasso()
las_model.fit(x_train,y_train)
y_pred_las = las_model.predict(x_test)
r2_score(y_test,y_pred_las)

0.994991989527428